In [2]:
!pip install pandas
!pip install numpy
!pip install pickle
!pip install fuzzywuzzy


You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import pickle
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/opt/conda/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Create address column 

In [4]:
#load address data 
address_data = pd.read_csv('../data/sc122a.txt', delimiter = '\t', header = 0, encoding='latin-1')

# Filter to only open schools OPEN in 2012-13 school year (so they probably have a working URL)
filtered_STATUS12 = (address_data['STATUS'] == 1) | (address_data['STATUS'] == 3) | (address_data['STATUS'] == 4) | (address_data['STATUS'] == 5) | (address_data['STATUS'] == 8)

address_data = address_data.loc[filtered_STATUS12].reset_index()
print(address_data.shape)
address_data.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,9,22,23,26,28,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(100413, 310)


,index,SURVYEAR,NCESSCH,FIPST,LEAID,SCHNO,STID,SEASCH,LEANM,SCHNAM,...,WHITE,WHALM,WHALF,PACIFIC,HPALM,HPALF,TR,TRALM,TRALF,TOTETH
0,0,2012,10000200277,1,100002,277,210,20,ALABAMA YOUTH SERVICES,SEQUOYAH SCH - CHALKVILLE CAMPUS,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,2012,10000201402,1,100002,1402,210,25,ALABAMA YOUTH SERVICES,EUFAULA SCH - EUFAULA CAMPUS,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2,2012,10000201667,1,100002,1667,210,50,ALABAMA YOUTH SERVICES,CAMPS,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,3,2012,10000201670,1,100002,1670,210,60,ALABAMA YOUTH SERVICES,DET CTR,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,4,2012,10000201705,1,100002,1705,210,30,ALABAMA YOUTH SERVICES,WALLACE SCH - MT MEIGS CAMPUS,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
street = address_data['LSTREE']
city = address_data['LCITY']
zipcode = list(map(str, address_data['LZIP'])) #convert all int in LZIP to strings so they can be appended 
state = address_data['LSTATE']

In [6]:
def make_address(street, city, state, zipcode): 
    '''Returns a new list of complete addresses with combined street, city, state, and zipcode 
    '''
    addresses = [] #empty list to fill with full addresses
    for i in np.arange(len(street)): 
        full_address = street[i].lower() + ' ' + city[i].lower() + ' ' + state[i].lower() + ' ' + zipcode[i].lower() #for every street in list of streets, append city, state, and zip to create full address
        addresses.append(full_address) #append full address into empty list of addresses
    return addresses 

In [7]:
full_address = make_address(street, city, state, zipcode) #list of full addresses 

In [8]:
full_address

['rt 2 old springville rd pinson al 36126',
 '315 outback rd clayton al 36016',
 'industrial school rd mt meigs al 36057',
 'industrial school rd mt meigs al 36057',
 'i85 service rd mount meigs al 36057',
 '8950 roebuck blvd birmingham al 35206',
 '1299 hillsboro parkway helena al 35080',
 '600 east alabama ave albertville al 35950',
 '402 east mccord ave albertville al 35950',
 '901 west mckinney ave albertville al 35950',
 '1100 horton rd albertville al 35950',
 '257 country club rd albertville al 35951',
 '1100 horton rd albertville al 35950',
 '6077 main st grant al 35747',
 '1990 asbury rd albertville al 35950',
 '140 claysville school rd guntersville al 35976',
 'hwy 75 douglas al 35964',
 'hwy 75 douglas al 35964',
 '2233 shoal creek rd arab al 35016',
 '6077 main st grant al 35747',
 '3685 union grove rd union grove al 35175',
 '12312 us hwy 431 south guntersville al 35976',
 '12312 us hwy 431 south guntersville al 35976',
 '200 bethlehem rd horton al 35950',
 '1050 scant city

In [9]:
address_data['ADDRESS13'] = full_address #add column of full addresses

In [10]:
#select only NCESSCH and ADRESS13 columns because we're joining on NCESSCH and we only want to add the ADDRESS13 values to charters_2015.pkl
address_NCESSCH_data = address_data[['NCESSCH', 'ADDRESS13']] 

In [11]:
address_NCESSCH_data 

,NCESSCH,ADDRESS13
0,10000200277,rt 2 old springville rd pinson al 36126
1,10000201402,315 outback rd clayton al 36016
2,10000201667,industrial school rd mt meigs al 36057
3,10000201670,industrial school rd mt meigs al 36057
4,10000201705,i85 service rd mount meigs al 36057
5,10000201706,8950 roebuck blvd birmingham al 35206
6,10000201876,1299 hillsboro parkway helena al 35080
7,10000500870,600 east alabama ave albertville al 35950
8,10000500871,402 east mccord ave albertville al 35950
9,10000500879,901 west mckinney ave albertville al 35950


## Match formatting of addresses and names in CER dataframe to formatting of new address data 

In [12]:
CER = pd.read_csv('CER_2012-13.csv', encoding = 'latin-1') #open CER file 

In [14]:
CER

,CER_NAME,CER_ADDRESS,CER_URL,CER_DESCRIPTION,CER_OPENED,CER_MEMBER,CER_GRADES,CER_GRADE_CATEGORIES
0,100 Academy of Excellence,"2341 Comstock Dr., N. Las Vegas, NV 89032",http://www.100academy.com/#,"Emphasis on Direct Instruction, college prep p...",2006,558,K-8,"Elementary-Middle,"
1,100 Black Men of the Bay Area Community School,"3400 Malcom Avenue, Oakland, CA 94605",http://www.the100schools.com/#,Focuses on males in STEM subjects. Will expand...,2012,92,"K-1, 4-6","Elementary-Middle,"
2,100 Legacy Academy Charter School,"75 Morton St., Newark, NJ 07193",http://www.100lacs.org/Pages/default.aspx#,"STEM, mentoring, extended school day. Will exp...",2012,300,6-8,"Middle,"
3,21st Century Charter School at Gary,"556 Washington St., Gary, IN 46402",http://www.21ccharter.org/#,"Offers character development, as well as art, ...",2005,372,K-12,"Elementary-Middle, Middle-High,"
4,21st Century Cyber Charter School,"805 Springdale Dr, Exton, PA 19355",http://www.21cccs.org/#,"Engages students in a highly motivational, stu...",2000,550,6-12,"Middle, Middle-High,"
5,21st Century Preparatory School,"1220 Mound Ave., Racine, WI 53404",http://www.21stprepschool.org/#,Core Knowledge Sequence and Direct Instruction.,2002,444,K-8,"Elementary-Middle,"
6,21st Century Public Academy,"3100 Menaul St., NE, Albuquerque, NM 87107",http://www.21stcenturypa.org/#,Hands-on learning. Students spend 1-2 weeks at...,2000,243,5-8,"Elementary-Middle,"
7,8 Points Charter School,"630 East State Street, Jacksonville, IL 62650",http://www.8pointscharter.org/#,"Prepare students academically, socially and em...",2011,70,5-7,"Elementary-Middle,"
8,"A T Jones Math, Science, and Technology Academy","4903 Ehrlich Rd., Tampa, FL 33624",http://www.atjonesacademy.com/flash.html#,"Math, science, technology emphasis.",2010,322,K-5,"Elementary,"
9,A+ Academy,"10327 Rylie Rd., Dallas, TX 75217",#,"Rigorous academics, combined with quality inst...",2001,970,PreK-12,"Pre Kindergarten, Elementary-Middle, Middle-H..."


In [15]:
#loop through every address, removing commas and periods, and converting all letters to lowercase
for i in np.arange(len(CER['CER_ADDRESS'])):
    CER.loc[i,'CER_ADDRESS'] = CER.loc[i,'CER_ADDRESS'].replace(',', '').replace('.','').lower() 


In [16]:
CER['CER_ADDRESS'] 

0                   2341 comstock dr n las vegas nv 89032
1                     3400 malcom avenue oakland ca 94605
2                            75 morton st newark nj 07193
3                         556 washington st gary in 46402
4                        805 springdale dr exton pa 19355
5                          1220 mound ave racine wi 53404
6                  3100 menaul st ne albuquerque nm 87107
7             630 east state street jacksonville il 62650
8                          4903 ehrlich rd tampa fl 33624
9                          10327 rylie rd dallas tx 75217
10                       1555 elaine rd columbus oh 43227
11                         100 obetz rd columbus oh 43207
12                         1201 n h st pensacola fl 32501
13                 710 e bella vista st lakeland fl 33805
14             22001 northwestern hwy southfield mi 48075
15                5701 red bird center dr dallas tx 75237
16                 6901 s westmoreland rd dallas tx 75237
17            

In [ ]:
#uniform formatting of name by making all characters lowercase and removing all punctuation
CER['CER_NAME'] = [str(name).lower() for name in CER['CER_NAME']]

## Add address column to nonduplicate charter dataframe

In [17]:
#load nonduplicate charter data 
charters_nonduplicate = pd.read_csv('../../nowdata/parsing/overlaps_removed_df.csv', sep = '\t', header = 0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (128,129,130,131,132,133,134,135,137,196,197,198,211,299,300,321,335) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
#merge addresses to charters_nonduplicate based on NCESSCH 
charters_nonduplicate_with_address = pd.merge(charters_nonduplicate, address_NCESSCH_data, how = 'left', on = 'NCESSCH') 
charters_nonduplicate_with_address = charters_nonduplicate_with_address.dropna(subset=['ADDRESS13'])

In [20]:
#match formatting for name and address 
charters_nonduplicate_with_address['SCHNAM12'] = [str(name).lower() for name in charters_nonduplicate_with_address['SCHNAM12']]

charters_nonduplicate_with_address['ADDRESS13'] = [str(address) for address in charters_nonduplicate_with_address['ADDRESS13']]

## Create new column of combined name+address to use as comparison with fuzzy ratio

In [26]:
#combine name and address for charters 
chartername = charters_nonduplicate_with_address['SCHNAM12']
charteraddress = charters_nonduplicate_with_address['ADDRESS13']
charters_nonduplicate_with_address['address_name_combined'] = [name+' '+address for name,address in zip(chartername, charteraddress)]
#charters_nonduplicate_with_address
                                                               
#combine name and address for CER 
CERname = CER['CER_NAME']
CERaddress = CER['CER_ADDRESS']
CER['address_name_combined'] = [name+' '+address for name,address in zip(CERname, CERaddress)]

In [ ]:
CER['address_name_combined']

## Using fuzzy ratio 

1. List fuzzy ratio between name+address columns in both data frames 
2. Keep the highest fuzzy ratio 
3. If fuzzy ratio is over 70, change the column name to its match 

In [ ]:

index = 0
for combined_charters in charters_nonduplicate_with_address['address_name_combined']:
    ratio = [fuzz.ratio(combined_charters, combinedCER) for combinedCER in CER['address_name_combined']]
    greatest_match_index = np.asarray(ratio).argmax()
    greatest_match_name = ratio[greatest_match_index]
    if greatest_match_name >= 70:
        print(str(index) + ' ' + 'name match found')
        charters_nonduplicate_with_address.loc[index, 'address_name_combined'] = CER.loc[greatest_match_index, 'address_name_combined']
    else:
        print(str(index))
    index = index + 1
        

0 name match found
1 name match found
2 name match found
3 name match found
4 name match found
5
6 name match found
7 name match found
8 name match found
9
10 name match found
11 name match found
12 name match found
13 name match found
14 name match found
15 name match found
16 name match found
17 name match found
18
19 name match found
20 name match found
21 name match found
22 name match found
23 name match found
24 name match found
25 name match found
26 name match found
27
28 name match found
29 name match found
30 name match found
31 name match found
32 name match found
33 name match found
34 name match found
35 name match found
36 name match found
37 name match found
38 name match found
39 name match found
40 name match found
41 name match found
42
43
44 name match found
45 name match found
46 name match found
47 name match found
48 name match found
49
50 name match found
51 name match found
52 name match found
53 name match found
54 name match found
55 name match found
56 name m

448 name match found
449
450 name match found
451
452 name match found
453
454 name match found
455 name match found
456 name match found
457 name match found
458 name match found
459 name match found
460 name match found
461 name match found
462 name match found
463 name match found
464 name match found
465 name match found
466 name match found
467 name match found
468 name match found
469 name match found
470 name match found
471
472 name match found
473 name match found
474
475 name match found
476 name match found
477 name match found
478 name match found
479 name match found
480 name match found
481
482 name match found
483 name match found
484
485
486 name match found
487 name match found
488 name match found
489 name match found
490
491 name match found
492 name match found
493
494 name match found
495 name match found
542 name match found
543 name match found
544 name match found
545
546 name match found
547 name match found
548 name match found
549
550 name match found
551 nam

952 name match found
953
954
955 name match found
956 name match found
957 name match found
958
959
960
961 name match found
962 name match found
963
964 name match found
965 name match found
966 name match found
967 name match found
968
969
970 name match found
971 name match found
972 name match found
973
974 name match found
975 name match found
976 name match found
977 name match found
978 name match found
979 name match found
980
981 name match found
982 name match found
983
984 name match found
985 name match found
986 name match found
987 name match found
988 name match found
989 name match found
990 name match found
991 name match found
992 name match found
993 name match found
994 name match found
995 name match found
996 name match found
997
998 name match found
999
1000 name match found
1001 name match found
1002 name match found
1003 name match found
1004 name match found
1005 name match found
1006 name match found
1007 name match found
1008 name match found
1009 name match

1387 name match found
1388 name match found
1389 name match found
1390 name match found
1391 name match found
1392 name match found
1393 name match found
1394 name match found
1395 name match found
1396 name match found
1397
1398 name match found
1399 name match found
1400 name match found
1401 name match found
1402 name match found
1403
1404
1405 name match found
1406 name match found
1407 name match found
1408 name match found
1409 name match found
1410 name match found
1411 name match found
1412 name match found
1413 name match found
1414 name match found
1415 name match found
1416 name match found
1417 name match found
1418 name match found
1419 name match found
1420 name match found
1421 name match found
1422
1423 name match found
1424 name match found
1425 name match found
1426 name match found
1427 name match found
1428 name match found
1429 name match found
1430 name match found
1431 name match found
1432 name match found
1433 name match found
1434 name match found
1435 name ma

1815 name match found
1816 name match found
1817
1818 name match found
1819
1820 name match found
1821 name match found
1822 name match found
1823 name match found
1824 name match found
1825
1826 name match found
1827 name match found
1828 name match found
1829 name match found
1830 name match found
1831 name match found
1832 name match found
1833 name match found
1834 name match found
1835 name match found
1836 name match found
1837
1838
1839 name match found
1840 name match found
1841 name match found
1842 name match found
1843
1844 name match found
1845 name match found
1846 name match found
1847 name match found
1848 name match found
1849 name match found
1850 name match found
1851 name match found
1852 name match found
1853 name match found
1854 name match found
1855 name match found
1856 name match found
1857 name match found
1858 name match found
1859 name match found
1860 name match found
1861 name match found
1862
1863 name match found
1864 name match found
1865 name match fou

2227 name match found
2228
2229 name match found
2230 name match found
2231 name match found
2232 name match found
2233 name match found
2234 name match found
2235 name match found
2236 name match found
2237 name match found
2238 name match found
2239 name match found
2240 name match found
2241 name match found
2242 name match found
2243 name match found
2244 name match found
2245 name match found
2246 name match found
2247 name match found
2248
2249 name match found
2250 name match found
2251 name match found
2252 name match found
2253 name match found
2254 name match found
2255 name match found
2256 name match found
2257 name match found
2258 name match found
2259 name match found
2260
2261 name match found
2262 name match found
2263 name match found
2264 name match found
2265 name match found
2266 name match found
2267 name match found
2268 name match found
2269 name match found
2270 name match found
2271 name match found
2272 name match found
2273 name match found
2274 name match f

## Merge URL with charter data 

In [ ]:
#merge based on combined name and address

charters_merge_CER = pd.merge(charters_nonduplicate_with_address, CER, how = 'left', left_on = ['address_name_combined'], right_on = ['address_name_combined']) 



In [ ]:
CER_columns = list(CER.columns) #list of all columns labels in CER
CER_columns.remove('CER_URL') #remove CER_URL from the list 

In [ ]:
charters_merge_CER = charters_merge_CER.drop(labels = CER_columns, axis = 1) #drop all columns in CER except CER_URL 

In [ ]:
len(charters_merge_CER)

## Drop NAN, empty, and 0 values 

In [ ]:
charters_merge_CER_cleaned = charters_merge_CER[charters_merge_CER['CER_URL'] != ''][charters_merge_CER['CER_URL'] != '0'].dropna(subset = ['CER_URL', 'SCHNAM12'])

In [ ]:
non_null_charter = charters_nonduplicate_with_address.dropna(subset = ['SCHNAM12'])

## Check URL matches

In [ ]:
#number of rows in charter data 
len(charters_nonduplicate_with_address)

In [ ]:
#number of rows in merged charter + URL data 
len(charters_merge_CER['NCESSCH']) #resulting merge has 6 more rows than original charters_nonduplicate

In [ ]:
#number of non-NAN, empty, or 0 value URLs + school name in merged data  
len(charters_merge_CER_cleaned)

In [ ]:
#number of non-NAN, empty, or 0 value URLs + school name in charter data 
len(non_null_charter)

In [ ]:
#number of unmatched schools 
len(non_null_charter) - len(charters_merge_CER_cleaned)

In [ ]:
non_null_charter['SCHNAM12']

In [ ]:
charters_merge_CER_cleaned['SCHNAM12']

In [ ]:
index=1
for schname in non_null_charter['SCHNAM12']: 
    if schname not in charters_merge_CER_cleaned['SCHNAM12']:
        print(schname + ' ' + str(index)) 
        index = index +1

In [ ]:
def common_data(list1, list2): 
    result = False
    for x in list1: 
        for y in list2: 
            if x = y: 
                result = True
                return result

In [ ]:
#number of non-NAN URL matches
non_null_vals = charters_merge_CER[charters_merge_CER['CER_URL'] != ''][charters_merge_CER['SCHNAM12'] != 'nan']
#non_null_vals = non_null_vals[non_null_vals['SCHNAM12'] ]
len(non_null_vals)

In [ ]:
non_null_vals['SCHNAM12']

In [ ]:
#number of unmatched schools 
len(non_null_charter) - len(non_null_vals)

# Why Unmatched URLS?
Unmatchated URLS(466) + Matched URLS(5707) = Total With Dropped NaN (6173) URLS. <br>
Question: Why could we not find match for 466 URLS? Possible reasons:
- Duplicate URLS? Perhaps a school may have multiple locations so those are all linked to the same website so it shows up multiple times in dataset?

In [ ]:
charters_merge_CER[['SCHNAM12', 'CER_URL']].dropna()

## Explore unmatched URLs

In [ ]:
#list of unmatched URLs 
index=1
for url in list(non_null_CER_URL): 
    if url not in list(non_null_vals):
        print(url + ' ' + str(index)) 
        index = index +1

In [ ]:
charters_nonduplicate_with_address['SCHNAM12'][charters_nonduplicate_with_address['SCHNAM12'].str.contains('academy of arts and academics')]

In [ ]:
charters_nonduplicate_with_address['ADDRESS13'][charters_nonduplicate_with_address['ADDRESS13'].str.contains('springfield or')]

In [ ]:
charters_merge_CER[charters_merge_CER['SCHNAM12'] == 'academy of arts and academics']

In [ ]:
CER['CER_URL'][CER['CER_URL'].str.contains('http://www.a3school.org')]

In [ ]:
CER[CER['CER_URL'] == 'http://www.a3school.org/#']

In [ ]:
CER.iloc[69:71]

In [ ]:
#charters_merge_CER.to_csv('../../nowdata/backups/charters_full_250_CER_URLs.pkl')